App that looks at the variation in the Return for the SP500 stock index.

In [1]:
import numpy as np
import panel as pn
import pandas as pd
import hvplot.pandas

import panel.widgets as pnw

import altair as alt
pn.extension('vega')
pn.extension()



ModuleNotFoundError: No module named 'hvplot'

In [ ]:
df = pd.read_csv("data/snp500.csv", parse_dates=["Date"])

In [7]:
snp = df.set_index("Date").sort_index()["Close"]

NameError: name 'df' is not defined

In [122]:
def compute_perc_change(snp, time_range = 100):
    
    shifted_snp = snp.shift(time_range)
    shifted_snp.name = "Close_lag"

    tmp = pd.concat([snp,shifted_snp], 1)
    tmp["Percent Change"] = ((tmp["Close"] - tmp["Close_lag"])/tmp["Close_lag"])*100
    
    percentile_outcomes = tmp["Percent Change"].quantile(np.arange(0, 1, 0.01))
    
    return tmp, percentile_outcomes

def plot_hist(tmp):
    return tmp.hvplot.hist("Percent Change", bins = 30, title="Distribution of Possible Returns")

def plot_perc_change(percentile_outcomes):
    t = percentile_outcomes.reset_index()
    t["Percentile"] = t.index
    perc_plot = (alt.Chart(t).mark_line(size=4)
                            .encode(x="Percentile",
                                    y=alt.X("Percent Change", scale = alt.Scale(domain=[-50, 150])),
                                    tooltip = ["Percentile", "Percent Change"]))
    return perc_plot.properties(width=650, height=300)

In [123]:
def plot_outcomes(time_range = 100):

    plots = []
    for i in range(10):
        x = df["Date"].sample(1).values[0]
        snp_sm = snp.loc[x:].iloc[:time_range]

        snp_sm = snp_sm.reset_index().reset_index()
        snp_sm["Close"] = snp_sm["Close"]/snp_sm["Close"].iloc[0]
        plt_1 = (alt.Chart(snp_sm)
                   .mark_line().encode(x="index",
                                       y=alt.X("Close", scale = alt.Scale(zero=False)))) 
        plots.append(plt_1)
        
    final_plot = alt.layer(*plots)
    #final_plot = plots[1]
    return final_plot

In [124]:
time_range = pn.widgets.IntSlider(start = 10, end=3000, 
                                  name='Days in the Market', value=10)

panelz = pn.Column(pn.Column('#S&P 500 Historical Returns Simulator', 
                             pn.WidgetBox(time_range)), 
                   pn.Row(None), 
                   pn.Row(None))

In [125]:
print(panelz)

Column
    [0] Column
        [0] Markdown(str)
        [1] WidgetBox(css_classes=['widget-box'])
            [0] IntSlider(end=3000, name='Days in the Market', start=10, value=10)
    [1] Row
        [0] Str(Empty)
    [2] Row
        [0] Str(Empty)


In [126]:
def update(event):
    outcomes, percentile_outcomes = compute_perc_change(snp, 
                                                        time_range = time_range.value)
    perc_plot = plot_perc_change(percentile_outcomes)
    panelz[2][0] = perc_plot
    panelz[1][0] = plot_hist(outcomes)
    #panelz[2] = plot_outcomes(time_range = time_range.value)
    

time_range.param.watch(update, "value")
panelz.servable()